In [ ]:
"""
    - https://huggingface.co/datasets/sonnetechnology/license-plate-text-recognition-full
    - https://huggingface.co/microsoft/Florence-2-large
    - https://huggingface.co/qualcomm/EasyOCR
"""

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image
from torchvision import transforms as T
from torch import nn
from torch.nn import functional as F
from torchvision.ops import sigmoid_focal_loss

In [4]:
import os
os.environ["HIP_VISIBLE_DEVICES"] = "1" # Keep this line for good measure, it might help with some internal ROCm ops.

import torch

print(f"HIP_VISIBLE_DEVICES (from os.environ): {os.environ.get('HIP_VISIBLE_DEVICES')}")
print(torch.cuda.is_available())
print(torch.cuda.device_count())

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")

    # THIS IS THE CRUCIAL CHANGE: Explicitly select device 1
    my_device = torch.device("cuda:1")
    print(f"\nUsing explicit device: {my_device}")

else:
    my_device = torch.device("cpu")
    print("\nCUDA/ROCm not available, falling back to CPU.")

HIP_VISIBLE_DEVICES (from os.environ): 1
True
1
Device 0: AMD Radeon RX 7800 XT

Using explicit device: cuda:1


In [ ]:
#  =================================== Dataset Images ===================================

In [5]:
# !pip install datasets
import datasets

ds = datasets.load_dataset("sonnetechnology/license-plate-text-recognition-full")

c:\Users\pc\Desktop\mis\unet_patent_detector\venv_unet\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\pc\Desktop\mis\unet_patent_detector\venv_unet\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pc\.cache\huggingface\hub\datasets--sonnetechnology--license-plate-text-recognition-full. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer M

In [ ]:
example = ds['train'][0]

In [ ]:
#  =================================== Florence 2 ===================================

In [ ]:
#  =================================== Easy OCR ===================================

In [ ]:
# pip install "qai-hub-models[easyocr]"
